# HEAL Monday Studies Board Update pipeline

Jupyter Notebook to follow the SOP for update the HEAL Monday Board. 
The notebook can be used for either step by step exploration, or running from a service like Google Colab.

In [ ]:
## If running on Google Colab, run this cell to mount Google Drive to access files on Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
## If running from Google Colab, might need to install this library
!pip install xlsxwriter

In [1]:
import sys
import logging
from pathlib import Path
sys.path.append('../scripts/')
import monday_board_update


In [2]:
## Set this to the directory where:
## 1- Monday Studies board has been exported.
## 2- All relevant tables from MySql database for HEAL have been exported as a csv to.
# input_dir = Path("/pat/to/data/dir")
input_dir = Path("/Users/hinashah/Documents/HEAL/MondayUpdate_July2025")

In [3]:
## Setup logger
logging.basicConfig(
        level=logging.DEBUG,
        format="%(asctime)s [%(levelname)s] %(message)s",
        filename= input_dir / "report-log.txt",
    )
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))


In [5]:
logging.info("---- STEP 1: Looking at Study Lookup Table")
gt_file = monday_board_update.import_study_lookup_table(input_dir)


---- STEP 1: Looking at Study Lookup Table
Number of entries in study lookup table: 1832
Number of distinct values in --appl_id--: 1812
---- NA count: 0
Number of distinct values in --xstudy_id--: 1444
---- NA count: 0
Number of distinct values in --study_most_recent_appl--: 1424
---- NA count: 0
Number of distinct values in --study_hdp_id--: 1340
---- NA count: 118
Number of distinct values in --study_hdp_id_appl--: 1320
---- NA count: 118


In [6]:
logging.info("---- STEP 2: Importing Monday Studies Board")
monday_board = monday_board_update.import_monday_board(input_dir) 


---- STEP 2: Importing Monday Studies Board
[PosixPath('/Users/hinashah/Documents/HEAL/MondayUpdate_July2025/HEAL_Studies_1753104562.xlsx')]
Index(['Name', 'Most Recent Appl_ID', 'HDP appl_ID', 'Project #', 'Archived',
       'HEAL-Related', 'Research Focus', 'Research Program',
       'Research Network', 'Title', 'Contact PI', 'Contact Email',
       'Administering IC', 'NIH PO', 'Institution(s)', 'PI(s)', 'Location',
       'Activity Code', 'Award Type', 'Award Year', 'Total Funded', 'Summary',
       'Project Start', 'Project End', 'Reporter Link', 'SBIR/STTR',
       'Data Engagement', 'Repo per Platform', 'Platform Reg Time',
       'CEDAR Form %', 'Creation Log', 'study_type',
       '"Get the Data" Engagement Board', 'VLMD Status', 'DD Tracker',
       'Checklist Exempt', 'Do not Engage', 'link to Data Dictionary Tracker',
       '_tmp_index'],
      dtype='object')
Number of records on Monday Board: 1488


In [6]:
logging.info("---- STEP 3: Compare lookup table and Monday Board")
mondayboard_missingin_lookup, lookup_fields = monday_board_update.compare_study_loookup_monday(gt_file, monday_board)

---- STEP 3: Compare lookup table and Monday Board
Number records from Monday already in lookup table: 1440
Number records from Monday that are not in lookup table (Consider these as discrepancies **Investigate**): 48
Number records from lookup table that are not on Monday (Potentially new entries): 0
Entries in Monday that are not in lookup table
                   Name Most Recent Appl_ID HDP appl_ID  \
1340           HDP01285                   -           -   
1341           HDP01286                   -           -   
1342           HDP01287                   -           -   
1343           HDP01288                   -           -   
1344           HDP01289                   -           -   
1345           HDP01290                   -           -   
1346           HDP01291                   -           -   
1347           HDP01292                   -           -   
1348           HDP01293                   -           -   
1349           HDP01294                   -           -   
1

In [ ]:
# import pandas as pd
# def get_unique_values(df:pd.DataFrame, col_name:str='appl_id'):
#     if col_name in df.columns:
#         return df[ ~pd.isna(df[col_name])][col_name].drop_duplicates()
#     return None

# convert_dict = {'appl_id':str}
# resnet_df = pd.read_csv(input_dir/"research_networks.csv", low_memory=False, dtype=convert_dict)
# logging.info(f"Research Network table has: {len(resnet_df)} entrie, with {len(get_unique_values(resnet_df))} appl_ids")
# print(resnet_df.columns)
# appl_ids = gt_file[['appl_id', 'study_most_recent_appl']].drop_duplicates()
# print(len(appl_ids))
# resnet_added = pd.merge(appl_ids, resnet_df[['appl_id', 'res_net']], how = 'left', left_on='appl_id', right_on='appl_id' )
# print(len(resnet_added))
# resnet_most_recent_appl_id = resnet_added[~pd.isna(resnet_added.res_net)][['study_most_recent_appl', 'res_net']]
# resnet_added_updated = pd.merge(appl_ids, resnet_most_recent_appl_id, how='left', left_on='study_most_recent_appl', right_on='study_most_recent_appl')
# resnet_added_updated.to_csv("/tmp/tmp_resnet_updated.csv", index=False)

Research Network table has: 2017 entrie, with 2017 appl_ids
Index(['appl_id', 'res_net', 'res_net_override_flag'], dtype='object')
1809
1809


In [ ]:
logging.info("---- STEP 4: Importing tables from MySQL and combining relevant information")
combined_data_ph1 = monday_board_update.import_mysql_data(input_dir, gt_file, monday_board, lookup_fields)

---- STEP 4: Importing tables from MySQL and combining relevant information
Awards table has: 2017 entries, with 2017 appl_ids
*** Combining the two reporter tables
2017
449
2466
Reporter table has: 2466 entries, with 2466 appl_ids
Platform generated table has: 1507 entries, with 1444 appl_ids
Platform table has 1444 unique HDP IDs
Repo mapping table has: 1326 entrie, with 1326 appl_ids
Research Network table has: 2017 entrie, with 2017 appl_ids
Engagment Flags table has: 1807 entrie, with 1807 appl_ids
--- Wrangling PI Emails
ALL PI emails associated with a project (identified by most_recent_appl)
      study_most_recent_appl               pi_email_latest
1                   9755001             kwatkins@rand.org
2                  10088639               damico@rand.org
3                  10996795         acfernan@med.umich.ed
4                  11001407               fqeadan@luc.edu
5                   9869480             LYNN.DEBAR@KP.ORG
...                     ...                  

In [ ]:
logging.info("---- STEP 5: Filling holes with MDS data")
combined_data_ph1 = monday_board_update.fill_in_holes_from_mds(input_dir, combined_data_ph1)

---- STEP 5: Filling holes with MDS data


In [ ]:
    ## Add CTN  data
logging.info("---- STEP 6: Adding any CTN data from MDS")
ctn_fields_platform = monday_board_update.get_ctndata_from_mds(input_dir)

---- STEP 6: Adding any CTN data from MDS
Number of CTN entries found in Platform MDS 44


/Users/hinashah/code/HEAL/metadata-sync/notebooks/../scripts/monday_board_update.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctn_data['project_title'] = ctn_data['project_title'].replace('0', '')


In [12]:
logging.info("---- STEP 7: Combining everything together")
all_data = monday_board_update.combine_mysql_ctn(combined_data_ph1, ctn_fields_platform)


---- STEP 7: Combining everything together
------------ Preview of the final combined dataset ---------------
     Activity Code Administering IC Archived Award Type  Award Year  \
0              U01            NIAAA     live          3      2019.0   
1              R34            NIAAA     live          3      2019.0   
2              R01            NIAAA     live          3      2020.0   
3              R34            NIAAA     live          1      2024.0   
4              R01            NIAAA     live          1      2024.0   
...            ...              ...      ...        ...         ...   
1318           NaN              NaN     live          0         NaN   
1498           NaN              NaN     live          0         NaN   
1499           NaN              NaN     live          0         NaN   
1500           NaN              NaN     live          0         NaN   
1501           NaN              NaN     live          0         NaN   

      CEDAR Form %  Checklist Exempt 

In [13]:
logging.info("---- STEP 8: Final Manipulation of all the data to make it Monday Board ready")
combined_data = monday_board_update.prepare_for_monday(all_data)

---- STEP 8: Final Manipulation of all the data to make it Monday Board ready
Counts for study types in the final dataset
study_type
HDP           1336
APPLIDONLY     104
CTN             44
Name: count, dtype: int64
Setting empty cells to  '-' in the following colulmns:
['Activity Code', 'Administering IC', 'Award Type', 'Contact Email', 'Contact PI', 'Institution(s)', 'NIH PO', 'PI(s)', 'Project #', 'Repo per Platform', 'Reporter Link', 'Research Focus', 'Research Network', 'Research Program', 'Summary', 'Title', 'key', 'HDP appl_ID', 'Most Recent Appl_ID']


In [14]:
logging.info("---- STEP 9: Final numbers and Export")
monday_board_update.export_finaldata(input_dir, combined_data, mondayboard_missingin_lookup, monday_board)  

---- STEP 9: Final numbers and Export
Number records from Monday already in final dataset: 1484
Number records from Monday that are not in lookup table (Consider these as discrepancies **Investigate**): 4
Number records from lookup table that are not on Monday (Potentially new entries): 0
****** Investigate/Delete the following entries on Monday that are not in there in the new Monday Excel upload
                   Name Most Recent Appl_ID HDP appl_ID  \
1388  10428343_HDP00882                 NaN         NaN   
1389  10488140_HDP00883                 NaN         NaN   
1390       9673173_none                 NaN         NaN   
1391       9769689_none                 NaN         NaN   

                                 Project # Archived HEAL-Related  \
1388             75N95020P00589-P00001-0-1      NaN          NaN   
1389  75N95019D00026-P00003-759501900088-1      NaN          NaN   
1390                       5U24HD095254-02      NaN          NaN   
1391                       5R01

In [13]:
mondayboard_missingin_lookup[~(mondayboard_missingin_lookup.study_type == 'CTN')].Name.values

array(['10428343_HDP00882', '10488140_HDP00883', '9673173_none',
       '9769689_none'], dtype=object)